<a href="https://colab.research.google.com/github/taiga0803/q-macro-assignment/blob/main/qmacro_pj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
# pwtから最新データ(vol10.1)の読み込み
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

#必要な国とデータの抽出
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'pop', 'avh', 'rkna', 'emp','rtfpna', 'labsh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

In [25]:
# Y=AK^(α)L^(1-α) の各変数の定義づけを行う
data['Y'] = data['rgdpna'] #GDP
data['alpha'] = 1 - data['labsh']  # 資本分配率
data['KL'] = (data['rkna']**data['alpha']) * (data['emp']**(1 - data['alpha'])) #K^(α)L^(1-α)
data['A'] = data['rgdpna'] / ((data['rkna']**data['alpha']) * (data['emp']**(1 - data['alpha']))) # A：技術資本

# 両辺をL(emp)で割り、y=Ak^αの形にし、同時に両辺対数を取る
data['logy'] = np.log(data['rgdpna'] / data['emp']) 
data['logk'] = np.log(data['KL'] / data['emp']) 
data['logA'] = np.log(data['A'])

#時間で微分するため、time diffeernceを取る
data['dt'] = data['year'].diff()  

# 対数差分をdtで割ることによって、時間で微分。
data['dlogy_dt'] = data['logy'].diff() / data['dt']
data['dlogk_dt'] = data['logk'].diff() / data['dt']
data['dlogA_dt'] = data['logA'].diff() / data['dt']

# 並べ替え、グループ化
data = data.sort_values('year')
grouped_data = data.groupby('countrycode')

# TFP成長率と資本深化率を計算
data['TFP_growth'] = data['dlogA_dt'] * 100  # TFP成長率
data['capital_deepening'] = data['dlogk_dt'] * 100  # 資本深化率
data['GDP_growth'] = data['dlogy_dt'] * 100  # 経済成長率（GDP成長率）

# 欠損値を削除
data = data.dropna()

# 各国平均を導出
summary = data.groupby('countrycode').agg({'GDP_growth': 'mean',
                                           'capital_deepening': 'mean',
                                           'TFP_growth': 'mean'})

# TFP寄与率と資本深化寄与率を計算
summary['TFP_share'] = summary['TFP_growth'] / summary['GDP_growth'] * 100
summary['capital_deepening_share'] = summary['capital_deepening'] / summary['GDP_growth'] * 100

print(summary)

             GDP_growth  capital_deepening  TFP_growth    TFP_share  \
countrycode                                                           
CAN            0.887700           0.239437    0.648264    73.027397   
DEU            0.662448           0.161122    0.501324    75.677399   
FRA            0.816542           0.603822    0.212720    26.051325   
GBR            1.036837           1.311810   -0.274969   -26.520018   
ITA           -0.045964           0.503274   -0.549236  1194.914917   
JPN            0.757615           0.036508    0.721106    95.181152   
USA            1.440635           0.605465    0.835166    57.972073   

             capital_deepening_share  
countrycode                           
CAN                        26.972775  
DEU                        24.322161  
FRA                        73.948730  
GBR                       126.520309  
ITA                     -1094.921265  
JPN                         4.818804  
USA                        42.027660  
